<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/master/WEAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WEAT Algorithm
## Test Statistic

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# returns s(w, A, B) for all w in W (passed as argument). Shape: n_words (in W) x 1
def swAB(W, A, B):
  #Calculate cosine-similarity between W and A, W and B
  WA = cosine_similarity(W,A)
  WB = cosine_similarity(W,B)
  #print('WA shape: ', WA.shape)
  #Take mean along columns
  WAmean = np.mean(WA, axis = 1)
  WBmean = np.mean(WB, axis = 1)
  
  #print('sWAB shape: ', WAmean.shape)
  
  return (WAmean - WBmean)
  
def test_statistic(X, Y, A, B):
  return (sum(swAB(X, A, B)) - sum(swAB(Y, A, B)))

def weat_effect_size(X, Y, A, B, embd):
  #Convert the set of words to matrix
  Xmat = np.array([embd[w] for w in X if w in embd])
  Ymat = np.array([embd[w] for w in Y if w in embd])
  Amat = np.array([embd[w] for w in A if w in embd])
  Bmat = np.array([embd[w] for w in B if w in embd])
  
  # Find X U Y
  XuY = list(set(X).union(Y))
  XuYmat = []
  for w in XuY:
    if w in embd:
      XuYmat.append(embd[w])
  XuYmat = np.array(XuYmat)
  print('X U Y Shape: ', XuYmat.shape)
  
  d = (np.mean(swAB(Xmat,Amat,Bmat)) - np.mean(swAB(Ymat,Amat,Bmat)))/np.std(swAB(XuYmat, Amat, Bmat))
  
  return d

## P-Value

In [0]:
import random
from itertools import combinations, filterfalse

def random_permutation(iterable, r=None):
  pool = tuple(iterable)
  r = len(pool) if r is None else r
  return tuple(random.sample(pool, r))

def weat_p_value(X, Y, A, B, embd, sample):
  size_of_permutation = min(len(X), len(Y))
  X_Y = X + Y
  test_stats_over_permutation = []
  
  Xmat = np.array([embd[w] for w in X if w in embd])
  Ymat = np.array([embd[w] for w in Y if w in embd])
  Amat = np.array([embd[w] for w in A if w in embd])
  Bmat = np.array([embd[w] for w in B if w in embd])
  
  if not sample:
      permutations = combinations(X_Y, size_of_permutation)
  else:
      permutations = [random_permutation(X_Y, size_of_permutation) for s in range(sample)]
      
  for Xi in permutations:
    Yi = filterfalse(lambda w:w in Xi, X_Y)
    Ximat = np.array([embd[w] for w in Xi if w in embd])
    Yimat = np.array([embd[w] for w in Yi if w in embd])
    test_stats_over_permutation.append(test_statistic(Ximat, Yimat, Amat, Bmat))
    
  unperturbed = test_statistic(Xmat, Ymat, Amat, Bmat)
  
  is_over = np.array([o > unperturbed for o in test_stats_over_permutation])
  print("All: ", test_stats_over_permutation)
  print("Unpertrubed: ", unperturbed)
  return is_over.sum() / is_over.size

## Test on sample input

In [36]:
X = ["bagpipe", "cello", "guitar", "lute", "trombone", "banjo", "clarinet", "harmonica", "mandolin", "trumpet", "bassoon", "drum", "harp", "oboe", "tuba", "bell", "fiddle", "harpsichord", "piano", "viola", "bongo",
"flute", "horn", "saxophone", "violin"] #Instruments
Y = ["arrow", "club", "gun", "missile", "spear", "axe", "dagger", "harpoon", "pistol", "sword", "blade", "dynamite", "hatchet", "rifle", "tank", "bomb", "firearm", "knife", "shotgun", "teargas", "cannon", "grenade",
    "mace", "slingshot", "whip"] #Weapons
A = ["caress", "freedom", "health", "love", "peace", "cheer", "friend", "heaven", "loyal", "pleasure", "diamond", "gentle", "honest", "lucky", "rainbow", "diploma", "gift", "honor", "miracle", "sunrise", "family",
    "happy", "laughter", "paradise", "vacation"] #Pleasant
B = ["abuse", "crash", "filth", "murder", "sickness", "accident", "death", "grief", "poison", "stink", "assault", "disaster", "hatred", "pollute", "tragedy", "divorce", "jail", "poverty", "ugly", "cancer", "kill", "rotten",
    "vomit", "agony", "prison"] #Unpleasant

#Load word embeddings
#load gensim formatted Full Glove embeddings
#!gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2

from gensim.models.keyedvectors import KeyedVectors

resourceFile = '/content/'


#glove = KeyedVectors.load_word2vec_format(resourceFile + 'gensim_glove.840B.300d.txt.bin', binary=True)
print('The glove embedding has been loaded!')

print('WEAT d = ', weat_effect_size(X, Y, A, B, glove))
print('WEAT p = ', weat_p_value(X, Y, A, B, glove, 1000))

The glove embedding has been loaded!
X U Y Shape:  (50, 300)
WEAT d =  1.5495627
All:  [-0.20551304146647453, 0.17732508853077888, -0.6206740848720074, 0.30140091851353645, 0.1181069128215313, -0.05647445097565651, -0.5253466106951237, -0.6503781788051128, -0.49709413573145866, 0.22090350463986397, -0.5853368081152439, -0.6247590892016888, 0.07627729699015617, 0.33030425384640694, -0.6417355574667454, -0.04906920716166496, 0.03531820699572563, -0.14018427953124046, 0.02485411986708641, -0.9952221773564816, -0.060898926109075546, 0.6022712178528309, 0.40501514449715614, -0.10907842591404915, 0.2976139672100544, 0.4821777977049351, -0.7193883024156094, 0.4565727300941944, 0.18758201226592064, -0.497607734054327, -0.24796677008271217, 0.3500887043774128, -0.28030603006482124, -0.02081439271569252, 0.616006750613451, -0.031252261251211166, 0.24622180685400963, -0.11388218030333519, -0.3213944025337696, 0.13040141388773918, -0.34662917628884315, 0.47327763214707375, 0.023144934326410294, -0

## WEAT with conceptor debiased embeddings

### Compute the conceptor matrix for all words and gender specific words.

In [0]:
#Compute the conceptor matrix
def post_process_cn_matrix(x, alpha = 2):
  print("starting...")
  #x = orig_embd.vectors
  print(x.shape)
  
  #Calculate the correlation matrix
  R = x.dot(x.T)/(x.shape[1])
  print("R calculated")
  
  #Calculate the conceptor matrix
  C = R @ (np.linalg.inv(R + alpha ** (-2) * np.eye(x.shape[0])))
  print("C calculated")
  
  #Calculate the negation of the conceptor matrix
  negC = np.eye(x.shape[0]) - C
  print("negC calculated")
  
  #Post-process the vocab matrix
  newX = (negC @ x).T
  print(newX.shape)
  return newX

Load all vectors from **glove**

In [28]:
#Load word embeddings
#download gensim formatted Full Glove embeddings
#!gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2

from gensim.models.keyedvectors import KeyedVectors

resourceFile = '/content/'

glove = KeyedVectors.load_word2vec_format(resourceFile + 'gensim_glove.840B.300d.txt.bin', binary=True)
print('The glove embedding has been loaded!')

The glove embedding has been loaded!


Load all vectors from **Word2Vec**

In [0]:
#load gensim formatted Full Word2vec embeddings
!gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
#!gunzip GoogleNews-vectors-negative300.bin.gz
  
import gensim

from gensim.models.keyedvectors import KeyedVectors

resourceFile = '/content/'

word2vec = KeyedVectors.load_word2vec_format(resourceFile + 'GoogleNews-vectors-negative300.bin', binary=True)
print('The word2vec embedding has been loaded!')

Load embeddings of all words from the ref. wordlist from a specific embedding

In [0]:
def load_all_vectors(embd, wikiWordsPath):
  all_words_index = {}
  all_words_mat = []
  with open(wikiWordsPath, "r+") as f_in:
    ind = 0
    for line in f_in:
      word = line.split(' ')[0]
      if word in embd:
        all_words_index[word] = ind
        all_words_mat.append(embd[word])
        ind = ind+1
        
  return all_words_index, all_words_mat

Conceptor all words and store it in a dictonary

In [30]:
import numpy as np

#!git clone https://github.com/PrincetonML/SIF
resourceFile = '/content/'
wikiWordsPath = resourceFile + '/SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

all_words_index, all_words_mat = load_all_vectors(glove, wikiWordsPath)
all_words_cn = post_process_cn_matrix(np.array(all_words_mat).T)

#Store all conceptored words in a dictonary
all_words = {}
for word, index in all_words_index.items():
  all_words[word] = all_words_cn[index,:]

starting...
(300, 128607)
R calculated
C calculated
negC calculated
(128607, 300)


Load embeddings of all gender-specific words (male_1, male_2, female_1, female_2) from a specific embedding

In [0]:
def load_gender_vectors(embd, gender_words):
  gender_embd_index = {}
  gender_embd_mat = []
  ind = 0
  for word in gender_words:
    if word in embd:
      gender_embd_index[word] = ind
      gender_embd_mat.append(embd[word])
      ind = ind+1
      
  return gender_embd_index, gender_embd_mat

Conceptor all gender words and store it in a dictonary

In [40]:
male = ["male", "man", "boy", "brother", "he", "him", "his", "son"]
female = ["female", "woman", "girl", "sister", "she", "her", "hers", "daughter"]
male_2 = ["brother", "father", "uncle", "grandfather", "son", "he", "his", "him"]
female_2 = ["sister", "mother", "aunt", "grandmother", "daughter", "she", "hers", "he"]
gender_words_list = male + female + male_2 + female_2

gender_words_index, gender_words_mat = load_gender_vectors(glove, gender_words_list)
gender_words_cn = post_process_cn_matrix(np.array(gender_words_mat).T)

#Store all conceptored words in a dictonary
#gender_words = {}
for word, index in gender_words_index.items():
  all_words[word] = gender_words_cn[index,:]

starting...
(300, 32)
R calculated
C calculated
negC calculated
(32, 300)


In [42]:
instruments = ["bagpipe", "cello", "guitar", "lute", "trombone", "banjo", "clarinet", "harmonica", "mandolin", "trumpet", "bassoon", "drum", "harp", "oboe", "tuba", "bell", "fiddle", "harpsichord", "piano", "viola", "bongo",
"flute", "horn", "saxophone", "violin"]
weapons = ["arrow", "club", "gun", "missile", "spear", "axe", "dagger", "harpoon", "pistol", "sword", "blade", "dynamite", "hatchet", "rifle", "tank", "bomb", "firearm", "knife", "shotgun", "teargas", "cannon", "grenade",
"mace", "slingshot", "whip"]
flowers = ["aster", "clover", "hyacinth", "marigold", "poppy", "azalea", "crocus", "iris", "orchid", "rose", "blue-bell", "daffodil", "lilac", "pansy", "tulip", "buttercup", "daisy", "lily", "peony", "violet", "carnation", "gladiola", "magnolia", "petunia", "zinnia"]
insects = ["ant", "caterpillar", "flea", "locust", "spider", "bedbug", "centipede", "fly", "maggot", "tarantula", "bee", "cockroach", "gnat", "mosquito", "termite", "beetle", "cricket", "hornet", "moth", "wasp", "blackfly",
"dragonfly", "horsefly", "roach", "weevil"]
pleasant = ["caress", "freedom", "health", "love", "peace", "cheer", "friend", "heaven", "loyal", "pleasure", "diamond", "gentle", "honest", "lucky", "rainbow", "diploma", "gift", "honor", "miracle", "sunrise", "family",
"happy", "laughter", "paradise", "vacation"]
unpleasant = ["abuse", "crash", "filth", "murder", "sickness", "accident", "death", "grief", "poison", "stink", "assault", "disaster", "hatred", "pollute", "tragedy", "divorce", "jail", "poverty", "ugly", "cancer", "kill", "rotten",
"vomit", "agony", "prison"]
math = ["math", "algebra", "geometry", "calculus", "equations", "computation", "numbers", "addition"]
arts = ["poetry", "art", "dance", "literature", "novel", "symphony", "drama", "sculpture"]
male = ["male", "man", "boy", "brother", "he", "him", "his", "son"]
female = ["female", "woman", "girl", "sister", "she", "her", "hers", "daughter"]
science = ["science", "technology", "physics", "chemistry", "Einstein", "NASA", "experiment", "astronomy"]
arts_2 = ["poetry", "art", "Shakespeare", "dance", "literature", "novel", "symphony", "drama"]
male_2 = ["brother", "father", "uncle", "grandfather", "son", "he", "his", "him"]
female_2 = ["sister", "mother", "aunt", "grandmother", "daughter", "she", "hers", "he"]
mental_disease = ["sad", "hopeless", "gloomy", "tearful", "miserable", "depressed"]
physical_disease = ["sick", "illness", "influenza", "disease", "virus", "cancer"]
temporary = ["impermanent", "unstable", "variable", "fleeting", "short-term", "brief", "occasional"]
permanent = ["stable", "always", "constant", "persistent", "chronic", "prolonged", "forever"]

print('WEAT d = ', weat_effect_size(science, arts + arts_2, male + male_2, female + female_2, all_words))
print('WEAT p = ', weat_p_value(science, arts + arts_2, male + male_2, female + female_2, all_words, 1000))

X U Y Shape:  (14, 300)
WEAT d =  0.8154110203417452
All:  [0.09453627505554689, 0.01764511262759779, -0.020737477012709754, 0.0014257417324020222, -0.011712191693806384, -0.014755409677973216, 0.0425616663529113, -0.09656315388255138, -0.013728973237668342, 0.12054966861554808, 0.0601908766497708, -0.011186326549796524, -0.10057436706801451, 0.08620679437513694, -0.05048083738046385, -0.11278531907393899, 0.07433486480313886, -0.08433851871541212, 0.00019622235737849075, -0.04474295442649506, -0.12630108241700783, 0.031239859474241585, -0.12965842497474014, 0.06606638535717005, 0.1118482288756654, -0.04584501571766899, -0.026656598343145127, 0.13041312174016637, -0.026630285294220013, 0.10397332918202376, 0.045166812076186745, 0.07829328438355845, -0.03261354898631132, 0.12285959901028375, -0.05090066780330154, -0.001152704116565595, 0.046806927432529365, 0.13478151670475283, -0.08927711704181888, 0.12226660848310705, 0.05986558237902143, 0.04253166514545263, 0.11042832960950692, -0.0

## WEAT algorithm from GITHUB gist
REF: https://gist.github.com/SandyRogers/e5c2e938502a75dcae25216e4fae2da5

In [0]:
class WEATTest(object):
    """
    Perform WEAT (Word Embedding Association Test) bias tests on a language model.
    Follows from Caliskan et al 2017 (10.1126/science.aal4230).
    """
    
    instruments = ["bagpipe", "cello", "guitar", "lute", "trombone", "banjo", "clarinet", "harmonica", "mandolin", "trumpet", "bassoon", "drum", "harp", "oboe", "tuba", "bell", "fiddle", "harpsichord", "piano", "viola", "bongo",
"flute", "horn", "saxophone", "violin"]
    weapons = ["arrow", "club", "gun", "missile", "spear", "axe", "dagger", "harpoon", "pistol", "sword", "blade", "dynamite", "hatchet", "rifle", "tank", "bomb", "firearm", "knife", "shotgun", "teargas", "cannon", "grenade",
    "mace", "slingshot", "whip"]
    flowers = ["aster", "clover", "hyacinth", "marigold", "poppy", "azalea", "crocus", "iris", "orchid", "rose", "blue-bell", "daffodil", "lilac", "pansy", "tulip", "buttercup", "daisy", "lily", "peony", "violet", "carnation", "gladiola", "magnolia", "petunia", "zinnia"]
    insects = ["ant", "caterpillar", "flea", "locust", "spider", "bedbug", "centipede", "fly", "maggot", "tarantula", "bee", "cockroach", "gnat", "mosquito", "termite", "beetle", "cricket", "hornet", "moth", "wasp", "blackfly",
    "dragonfly", "horsefly", "roach", "weevil"]
    pleasant = ["caress", "freedom", "health", "love", "peace", "cheer", "friend", "heaven", "loyal", "pleasure", "diamond", "gentle", "honest", "lucky", "rainbow", "diploma", "gift", "honor", "miracle", "sunrise", "family",
    "happy", "laughter", "paradise", "vacation"]
    unpleasant = ["abuse", "crash", "filth", "murder", "sickness", "accident", "death", "grief", "poison", "stink", "assault", "disaster", "hatred", "pollute", "tragedy", "divorce", "jail", "poverty", "ugly", "cancer", "kill", "rotten",
    "vomit", "agony", "prison"]
    math = ["math", "algebra", "geometry", "calculus", "equations", "computation", "numbers", "addition"]
    arts = ["poetry", "art", "dance", "literature", "novel", "symphony", "drama", "sculpture"]
    male = ["male", "man", "boy", "brother", "he", "him", "his", "son"]
    female = ["female", "woman", "girl", "sister", "she", "her", "hers", "daughter"]
    science = ["science", "technology", "physics", "chemistry", "Einstein", "NASA", "experiment", "astronomy"]
    arts_2 = ["poetry", "art", "Shakespeare", "dance", "literature", "novel", "symphony", "drama"]
    male_2 = ["brother", "father", "uncle", "grandfather", "son", "he", "his", "him"]
    female_2 = ["sister", "mother", "aunt", "grandmother", "daughter", "she", "hers", "he"]
    mental_disease = ["sad", "hopeless", "gloomy", "tearful", "miserable", "depressed"]
    physical_disease = ["sick", "illness", "influenza", "disease", "virus", "cancer"]
    temporary = ["impermanent", "unstable", "variable", "fleeting", "short-term", "brief", "occasional"]
    permanent = ["stable", "always", "constant", "persistent", "chronic", "prolonged", "forever"]
    
    def __init__(self, model):
        """Setup a Word Embedding Association Test for a given spaCy language model.
        
        EXAMPLE:
            >>> nlp = spacy.load('en_core_web_md')
            >>> test = WEATTest(nlp)
            >>> test.run_test(WEATTest.instruments, WEATTest.weapon, WEATTest.pleasant, WEATTest.unpleasant)
        """
        self.model = model

    @staticmethod
    def word_association_with_attribute(self, w, A, B):
        return np.mean([cosine_similarity(np.array(w).reshape(1,-1),np.array(a).reshape(1,-1)) for a in A]) - np.mean([cosine_similarity(np.array(w).reshape(1,-1),np.array(b).reshape(1,-1)) for b in B])

    @staticmethod
    def differential_assoication(self, X, Y, A, B):
        return np.sum([self.word_association_with_attribute(self, x, A, B) for x in X]) - np.sum([self.word_association_with_attribute(self, y, A, B) for y in Y])

    @staticmethod
    def weat_effect_size(self, X, Y, A, B):
        return (
            np.mean([self.word_association_with_attribute(self, x, A, B) for x in X]) -
            np.mean([self.word_association_with_attribute(self, y, A, B) for y in Y])
        ) / np.std([self.word_association_with_attribute(self, w, A, B) for w in X + Y])

    @staticmethod
    def random_permutation(self, iterable, r=None):
        pool = tuple(iterable)
        r = len(pool) if r is None else r
        return tuple(random.sample(pool, r))

    @staticmethod
    def weat_p_value(self, X, Y, A, B, sample):
        size_of_permutation = min(len(X), len(Y))
        X_Y = X + Y
        observed_test_stats_over_permutations = []

        if not sample:
            permutations = combinations(X_Y, size_of_permutation)
        else:
            permutations = [self.random_permutation(self, X_Y, size_of_permutation) for s in range(sample)]
        print(np.array(X_Y).shape)
        for Xi in permutations:
            Yi = filterfalse(lambda w:w in Xi, X_Y)
            observed_test_stats_over_permutations.append(self.differential_assoication(self, Xi, Yi, A, B))

        unperturbed = self.differential_assoication(self, X, Y, A, B)
        is_over = np.array([o > unperturbed for o in observed_test_stats_over_permutations])
        return is_over.sum() / is_over.size

    @staticmethod
    def weat_stats(X, Y, A, B, self, sample_p=None):
        test_statistic = self.differential_assoication(self, X, Y, A, B)
        effect_size = self.weat_effect_size(self, X, Y, A, B)
        p = self.weat_p_value(self, X, Y, A, B, sample=sample_p)
        return test_statistic, effect_size, p

    def run_test(self, target_1, target_2, attributes_1, attributes_2, sample_p=None):
        """Run the WEAT test for differential association between two 
        sets of target words and two seats of attributes.
        
        EXAMPLE:
            >>> test.run_test(WEATTest.instruments, WEATTest.weapon, WEATTest.pleasant, WEATTest.unpleasant)
            >>> test.run_test(a, b, c, d, sample_p=1000) # use 1000 permutations for p-value calculation
            >>> test.run_test(a, b, c, d, sample_p=None) # use all possible permutations for p-value calculation
            
        RETURNS:
            (d, e, p). A tuple of floats, where d is the WEAT Test statistic, 
            e is the effect size, and p is the one-sided p-value measuring the
            (un)likeliness of the null hypothesis (which is that there is no
            difference in association between the two target word sets and
            the attributes).
            
            If e is large and p small, then differences in the model between 
            the attribute word sets match differences between the targets.
        """
        X = [list(self.model[w]) for w in target_1]
        Y = [list(self.model[w]) for w in target_2]
        A = [list(self.model[w]) for w in attributes_1]
        B = [list(self.model[w]) for w in attributes_2]
        print(X)
        return self.weat_stats(X, Y, A, B, self, sample_p)


## Code test

In [0]:
#nlp = spacy.load('glove')
test = WEATTest(glove)
test.run_test(WEATTest.instruments, WEATTest.weapons, WEATTest.pleasant, WEATTest.unpleasant, 1000)

The glove embedding has been loaded!
X Shape:  (3, 300)
Y Shape:  (3, 300)
A Shape:  (2, 300)
B Shape:  (2, 300)
X U Y Shape:  (6, 300)
WA shape:  (3, 2)
sWAB shape:  (3,)
WA shape:  (3, 2)
sWAB shape:  (3,)
WA shape:  (6, 2)
sWAB shape:  (6,)
WEAT d =  1.8613524
